In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt


In [5]:
def build_road_network_spatial_index(edges_filepath):
    """
    从CSV文件加载路网数据并构建一个R-tree空间索引。

    Args:
        edges_filepath (str): 包含路网边的CSV文件路径。
                              文件需要一个名为 'geometry' 的列，其内容为WKT格式的几何图形。

    Returns:
        GeoDataFrame: 包含路网数据和空间索引的GeoDataFrame。
    """
    # 使用pandas加载CSV文件
    edges_df = pd.read_csv(edges_filepath)

    # 将WKT格式的'geometry'列转换为Shapely几何对象
    # GeoPandas可以自动识别WKT字符串并进行转换
    try:
        edges_df['geometry'] = edges_df['geometry'].apply(wkt.loads)
    except Exception as e:
        print(f"转换WKT时出错: {e}")
        print("请确保 'geometry' 列是有效的WKT格式。")
        return None

    # 创建GeoDataFrame
    # 假设原始数据是WGS84 (EPSG:4326)，这是GPS的常用坐标系
    road_network_gdf = gpd.GeoDataFrame(edges_df, geometry='geometry', crs='epsg:4326')

    # 为了进行精确的米制单位缓冲（buffer）操作，将坐标系转换为一个投影坐标系
    # 这里我们使用一个通用的世界墨卡托投影 (EPSG:3857)
    # 在特定区域，使用区域性的UTM投影会更精确
    print("正在转换坐标系以进行精确的距离计算...")
    road_network_gdf = road_network_gdf.to_crs(epsg=3857)
    print("坐标系转换完成。")

    # Geopandas会在首次访问 .sindex 属性时自动构建R-tree索引
    print("正在构建路网的R-tree空间索引...")
    _ = road_network_gdf.sindex
    print("空间索引构建完成。")

    return road_network_gdf


def find_candidate_edges(gps_point, road_network_gdf, search_radius_meters=20):
    """
    为一个GPS点查找在指定搜索半径内的候选路段。

    Args:
        gps_point (Point): 一个Shapely的Point对象，代表GPS点。
        road_network_gdf (GeoDataFrame): 包含路网数据的GeoDataFrame。
        search_radius_meters (int): 搜索半径（单位：米）。

    Returns:
        list: 候选路段的ID列表 (这里使用'osmid')。
    """
    # 以GPS点为中心创建一个圆形缓冲区
    # 由于GeoDataFrame已经是米制单位的投影坐标系，所以可以直接使用半径
    search_buffer = gps_point.buffer(search_radius_meters)

    # 使用空间索引进行第一次粗略筛选，找出与缓冲区边界框相交的路段
    possible_matches_index = list(road_network_gdf.sindex.intersection(search_buffer.bounds))
    possible_matches = road_network_gdf.iloc[possible_matches_index]

    # 进行第二次精确筛选，找出实际与缓冲区相交的路段
    precise_matches = possible_matches[possible_matches.intersects(search_buffer)]

    # 返回候选路段的 'osmid' 列表
    return precise_matches['osmid'].tolist()

In [6]:
road_network_filepath = 'road_network_edges.csv'
gps_trace_filepath = 'original_trajectories_under_70m_diff.csv'
output_filepath = 'gps_candidate_road_segments.csv'
search_radius = 30  # 搜索半径（米），可根据需求调整

# --- 2. 加载路网并构建空间索引 ---
print("步骤 1: 加载路网数据并构建空间索引...")
road_network_gdf = build_road_network_spatial_index(road_network_filepath)


# --- 3. 加载GPS轨迹数据 ---
print("步骤 2: 加载GPS轨迹数据...")
try:
    # 假设GPS数据文件有 'latitude' 和 'longitude' 两列
    gps_df = pd.read_csv(gps_trace_filepath)
    # 将其转换为GeoDataFrame
    gps_gdf = gpd.GeoDataFrame(
        gps_df,
        geometry=gpd.points_from_xy(gps_df.longitude, gps_df.latitude),
        crs='epsg:4326'  # 假设是WGS84坐标系
    )
    # 将GPS数据也转换为与路网相同的米制投影坐标系
    gps_gdf = gps_gdf.to_crs(road_network_gdf.crs)
    print(f"成功加载 {len(gps_gdf)} 个GPS点。")
except FileNotFoundError:
    print(f"错误: GPS文件未找到于 '{gps_trace_filepath}'")

except Exception as e:
    print(f"加载GPS文件时出错: {e}")
    print("请确保文件包含 'latitude' 和 'longitude' 列。")



# --- 4. 为每个GPS点查找候选路段 ---
print(f"步骤 3: 为每个GPS点查找半径为 {search_radius} 米的候选路段...")
results = []

# 为了方便，给每个GPS点一个唯一的ID
if 'gps_id' not in gps_gdf.columns:
    gps_gdf['gps_id'] = range(len(gps_gdf))

for index, gps_point_row in gps_gdf.iterrows():
    gps_id = gps_point_row['gps_id']
    gps_geom = gps_point_row['geometry']

    # 查找候选路段
    candidate_osmids = find_candidate_edges(gps_geom, road_network_gdf, search_radius)

    # 将结果保存下来
    results.append({
        'gps_id': gps_id,
        'latitude': gps_point_row.get('latitude'),  # .get() 避免列不存在时出错
        'longitude': gps_point_row.get('longitude'),
        'candidate_road_osmids': candidate_osmids
    })

    # 打印进度
    if (index + 1) % 100 == 0:
        print(f"已处理 {index + 1}/{len(gps_gdf)} 个GPS点...")

print("所有GPS点的候选路段查找完成。")
print("-" * 30)

# --- 5. 保存结果到CSV文件 ---
print(f"步骤 4: 将结果写入 '{output_filepath}'...")
results_df = pd.DataFrame(results)
results_df.to_csv(output_filepath, index=False)
print("处理完成！")

步骤 1: 加载路网数据并构建空间索引...
正在转换坐标系以进行精确的距离计算...
坐标系转换完成。
正在构建路网的R-tree空间索引...
空间索引构建完成。
步骤 2: 加载GPS轨迹数据...


/tmp/ipykernel_378208/981083981.py:13: DtypeWarning: Columns (14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(edges_filepath)


成功加载 218844 个GPS点。
步骤 3: 为每个GPS点查找半径为 30 米的候选路段...
已处理 100/218844 个GPS点...
已处理 200/218844 个GPS点...
已处理 300/218844 个GPS点...
已处理 400/218844 个GPS点...
已处理 500/218844 个GPS点...
已处理 600/218844 个GPS点...
已处理 700/218844 个GPS点...
已处理 800/218844 个GPS点...
已处理 900/218844 个GPS点...
已处理 1000/218844 个GPS点...
已处理 1100/218844 个GPS点...
已处理 1200/218844 个GPS点...
已处理 1300/218844 个GPS点...
已处理 1400/218844 个GPS点...
已处理 1500/218844 个GPS点...
已处理 1600/218844 个GPS点...
已处理 1700/218844 个GPS点...
已处理 1800/218844 个GPS点...
已处理 1900/218844 个GPS点...
已处理 2000/218844 个GPS点...
已处理 2100/218844 个GPS点...
已处理 2200/218844 个GPS点...
已处理 2300/218844 个GPS点...
已处理 2400/218844 个GPS点...
已处理 2500/218844 个GPS点...
已处理 2600/218844 个GPS点...
已处理 2700/218844 个GPS点...
已处理 2800/218844 个GPS点...
已处理 2900/218844 个GPS点...
已处理 3000/218844 个GPS点...
已处理 3100/218844 个GPS点...
已处理 3200/218844 个GPS点...
已处理 3300/218844 个GPS点...
已处理 3400/218844 个GPS点...
已处理 3500/218844 个GPS点...
已处理 3600/218844 个GPS点...
已处理 3700/218844 个GPS点...
已处理 3800/218844 个GPS点...
已处理 3900

In [7]:
import random
def visualize_single_order_candidates(gps_df, road_network_gdf, search_radius):
    """
    随机抽取一个order_id，提取其GPS轨迹和所有候选路段，并保存以便于QGIS可视化。

    Args:
        gps_df (DataFrame): 包含所有GPS点的原始Pandas DataFrame。
        road_network_gdf (GeoDataFrame): 包含路网数据的GeoDataFrame（已投影）。
        search_radius (int): 搜索半径（米）。
    """


    if 'order_id' not in gps_df.columns:
        print("错误: 'gps_trace.csv' 中未找到 'order_id' 列。跳过可视化步骤。")
        return

    # 1. 随机选择一个 order_id
    unique_orders = gps_df['order_id'].unique()
    if len(unique_orders) == 0:
        print("警告: GPS数据中没有找到任何 order_id。")
        return
    selected_order_id = random.choice(unique_orders)
    print(f"随机抽取的订单ID为: {selected_order_id}")

    # 2. 筛选出该订单的所有GPS点
    single_order_gps_df = gps_df[gps_df['order_id'] == selected_order_id].copy()
    print(f"该订单包含 {len(single_order_gps_df)} 个GPS点。")

    # 将这些GPS点转换为与路网相同的投影坐标系
    single_order_gdf = gpd.GeoDataFrame(
        single_order_gps_df,
        geometry=gpd.points_from_xy(single_order_gps_df.longitude, single_order_gps_df.latitude),
        crs='epsg:4326'
    ).to_crs(road_network_gdf.crs)

    # 3. 为该订单的所有GPS点查找候选路段
    all_candidate_osmids = set()
    for _, gps_row in single_order_gdf.iterrows():
        candidates = find_candidate_edges(gps_row['geometry'], road_network_gdf, search_radius)
        all_candidate_osmids.update(candidates)

    print(f"为整个轨迹找到了 {len(all_candidate_osmids)} 个唯一的候选路段。")

    if not all_candidate_osmids:
        print("未找到任何候选路段，无法生成可视化文件。")
        return

    # 4. 从原始路网中提取这些候选路段的完整信息
    candidate_roads_gdf = road_network_gdf[road_network_gdf['osmid'].isin(all_candidate_osmids)].copy()

    # 为了方便在QGIS中查看，将几何坐标转换回WGS84 (EPSG:4326)
    candidate_roads_gdf_wgs84 = candidate_roads_gdf.to_crs(epsg=4326)

    # 将Shapely几何对象转换为WKT文本格式，以便存入CSV
    candidate_roads_gdf_wgs84['geometry'] = candidate_roads_gdf_wgs84['geometry'].apply(lambda geom: geom.wkt)

    # 5. 保存结果到两个独立的CSV文件
    # 文件名中包含order_id，方便识别
    gps_output_path = f"visualization_gps_points_order_{selected_order_id}.csv"
    roads_output_path = f"visualization_candidate_roads_order_{selected_order_id}.csv"

    single_order_gps_df.to_csv(gps_output_path, index=False)
    candidate_roads_gdf_wgs84.to_csv(roads_output_path, index=False)

    print("-" * 30)
    print("可视化文件已保存！")
    print(f"GPS轨迹点: {gps_output_path}")
    print(f"候选路段: {roads_output_path}")
    print("\n如何在QGIS中加载:")
    print("  1. 打开QGIS，选择'图层' -> '添加图层' -> '添加分隔文本图层...'。")
    print(f"  2. 对于GPS点文件 '{gps_output_path}':")
    print("     - X字段选'longitude', Y字段选'latitude'。")
    print(f"  3. 对于路段文件 '{roads_output_path}':")
    print("     - '几何图形定义'部分，选择'WKT(Well-known text)'，几何图形字段选'geometry'。")
    print("  4. 确保两个图层的坐标参考系(CRS)都设置为 'EPSG:4326 - WGS 84'。")

In [8]:
visualize_single_order_candidates(gps_df, road_network_gdf, search_radius)


步骤 5: 为单个随机订单生成可视化文件
随机抽取的订单ID为: 041556197a293a0b6a762df3831877e0
该订单包含 55 个GPS点。
为整个轨迹找到了 35 个唯一的候选路段。
------------------------------
可视化文件已保存！
GPS轨迹点: visualization_gps_points_order_041556197a293a0b6a762df3831877e0.csv
候选路段: visualization_candidate_roads_order_041556197a293a0b6a762df3831877e0.csv

如何在QGIS中加载:
  1. 打开QGIS，选择'图层' -> '添加图层' -> '添加分隔文本图层...'。
  2. 对于GPS点文件 'visualization_gps_points_order_041556197a293a0b6a762df3831877e0.csv':
     - X字段选'longitude', Y字段选'latitude'。
  3. 对于路段文件 'visualization_candidate_roads_order_041556197a293a0b6a762df3831877e0.csv':
     - '几何图形定义'部分，选择'WKT(Well-known text)'，几何图形字段选'geometry'。
  4. 确保两个图层的坐标参考系(CRS)都设置为 'EPSG:4326 - WGS 84'。


/tmp/ipykernel_378208/622965636.py:57: UserWarning: Geometry column does not contain geometry.
  candidate_roads_gdf_wgs84['geometry'] = candidate_roads_gdf_wgs84['geometry'].apply(lambda geom: geom.wkt)
